#Global Climate Trends
##Analysis

In [1]:
import pandas as pd
import os
import csv

In [2]:
# Path to folders
input_folder_path = os.path.join(".", "input_data")
output_folder_path = os.path.join(".", "output_data")

# Path to input files
pec_input_file_path = os.path.join(input_folder_path, "primary_energy_consumption.csv")

# Path to output files
pec_output_file_path = os.path.join(output_folder_path, "primary_energy_consumption_by_year_n_country.csv")

In [3]:
def process_file(filename, columname):
    input_file_path = os.path.join(input_folder_path, filename)
    output_file_path = os.path.join(input_folder_path, filename)

    # Read data and store it in a dataframe
    data_df = pd.read_csv(input_file_path)

    df_array = []

    # Loop over the columns of the dataframe skipping the first column
    for column in data_df.columns[1:]:
        # Create a new dataframe with the country and the year
        cy_list = data_df[['Country', column]].copy()

        # Adding a new column to the dataframe and positionong
        cy_list.insert(0, 'Year', column)

        #Rename for dataset
        cy_list.rename(columns={column: columname}, inplace=True)

        # Adding the new dataframe to the array
        df_array.append(cy_list)


    # Merge dfs 
    merged_df = pd.concat(df_array, ignore_index=True)

    #Save csv
    merged_df.to_csv(output_file_path, index=False, header=True)
    return merged_df

In [4]:
pec_df = process_file("primary_energy_consumption.csv", "Primary Energy Consumption")
pec_df

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

Energy Consumption Per Capita. DF

In [ ]:
ecpc_df = process_file("energy_consumption_per_capita.csv", "Energy Consumption per Capita")
ecpc_df

InvalidIndexError: Reindexing only valid with uniquely valued Index objects